In [1]:
# if running in Google Colab, install required packages and enable widget manager
try:
    import warnings
    import logging
    from google.colab import output

    # Install daspi and ipywidgets in Colab
    !pip install daspi ipywidgets --quiet

    # Suppress matplotlib font warnings in Colab
    warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")
    logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

    output.enable_custom_widget_manager()
except ModuleNotFoundError:
    pass

In [43]:
import numpy as np
import daspi as dsp
import pandas as pd

from math import pi
from typing import Tuple

from ipywidgets import interact
from ipywidgets import FloatSlider

N_SAMPLES = 500
DURATION = 2*pi

def calculate_sine_wave(
        phase: float, duration: float, n_samples: int
        ) -> Tuple[np.ndarray, np.ndarray]:
    """Calculate a sine wave with the given phase, duration, and number 
    of samples.
    
    The sine wave is calculated using the formula:
    y(t) = sin(2 * pi * t + phase)

    Parameters
    ----------
    phase : float
        The phase of the sine wave in radians.
    duration : float
        The duration of the sine wave in seconds.
    n_samples : int
        The number of samples to generate.
    
    Returns
    -------
    Tuple[np.ndarray, np.ndarray]
        A tuple containing the time array and the calculated sine wave
        as NumPy arrays.
    """
    t = np.linspace(0, duration, n_samples, endpoint=False)
    return t, np.sin(t + phase)

def calculate_cosine_wave(
        phase: float, duration: float, n_samples: int
        ) -> Tuple[np.ndarray, np.ndarray]:
    """Calculate a cosine wave with the given phase, duration, and 
    number of samples.

    The cosine wave is calculated using the formula:
    y(t) = cos(2 * pi * t + phase)

    Parameters
    ----------
    phase : float
        The phase of the cosine wave in radians.
    duration : float
        The duration of the cosine wave in seconds.
    n_samples : int
        The number of samples to generate.
    
    Returns
    -------
    Tuple[np.ndarray, np.ndarray]
        A tuple containing the time array and the calculated cosine wave
        as NumPy arrays.
    """
    t = np.linspace(0, duration, n_samples, endpoint=False)
    return t, np.cos(t + phase)

def phase_from_correlation_coefficient(correlation_coefficient: float) -> float:
    """Calculate the phase difference in radians from the correlation 
    coefficient between two signals.

    The phase difference is calculated using the formula:
    phase = arccos(correlation_coefficient)

    Parameters
    ----------
    correlation_coefficient : float
        The correlation coefficient between two signals, which should be 
        in the range [-1, 1].
    
    Returns
    -------
    float
        The phase difference in radians, which will be in the range [0, pi].
    """
    assert -1 <= correlation_coefficient <= 1, (
        'Correlation coefficient must be in the range [-1, 1]')
    return np.arccos(correlation_coefficient)

def plot_correlation_coefficient(correlation_coefficient: float) -> None:
    phase=phase_from_correlation_coefficient(correlation_coefficient)
    t, sine_wave = calculate_sine_wave(
        phase=phase, duration=DURATION, n_samples=N_SAMPLES)
    t, cosine_wave = calculate_cosine_wave(
        phase=phase, duration=DURATION, n_samples=N_SAMPLES)
    t, sine_wave_base = calculate_sine_wave(
        phase=0, duration=DURATION, n_samples=N_SAMPLES)
    amplitude = np.sin(phase)
    df_wave = pd.DataFrame({
        'Zeit': t,
        'Sinus': sine_wave,
        'Cosinus': cosine_wave,
        'Scatter_x': sine_wave_base + amplitude * np.random.normal(0, 0.4, N_SAMPLES),
        'Scatter_y': sine_wave + amplitude * np.random.normal(0, 0.4, N_SAMPLES),})
    
    df_r = pd.DataFrame({
        'x': [0, phase],
        'y': [1, 1],
    })

    if phase > pi/2:
        df_sides = pd.DataFrame({
            'x_sin': [pi, phase],
            'y_sin': [abs(np.cos(phase)), 1],
            'x_cos': [0, pi],
            'y_cos': [0, abs(np.cos(phase))],
        })
    else:
        df_sides = pd.DataFrame({
            'x_sin': [0, phase],
            'y_sin': [np.cos(phase), 1],
            'x_cos': [0, 0],
            'y_cos': [0, np.cos(phase)],})

    df_angle = pd.DataFrame({
        'x': np.linspace(0, phase, N_SAMPLES),
        'y': [0.2] * N_SAMPLES})

    chart = dsp.JointChart(
            source=df_wave,
            target=('', 'Sinus', 'Zeit', 'Scatter_y'),
            feature=('', 'Zeit', 'Cosinus', 'Scatter_x'),
            nrows=2,
            ncols=2,
            figsize=(5, 5),
            dpi=300,
        ).plot(
            dsp.SkipSubplot
        ).plot(
            dsp.Line,
            hide_axis='feature',
            visible_spines='target',
            color=dsp.COLOR.BAD,
        ).plot(
            dsp.Line,
            hide_axis='target',
            visible_spines='feature',
            color=dsp.COLOR.GOOD,
        ).plot(
            dsp.LinearRegressionLine,
            show_scatter=True,
            show_fit_ci=True,
            show_pred_ci=True,
        ).label(
            axes_titles=('', 'Sinus', 'Cosinus', 'Korrelationskoeffizient R = {:.2f}'.format(np.corrcoef(df_wave['Scatter_y'], df_wave['Scatter_x'])[0, 1])),
        )
    chart.figure.delaxes(chart.axes[0])
    chart.axes[1].set(xlim=(0, DURATION), ylim=(-1.1, 1.1))
    chart.axes[2].set(xlim=(-1.1, 1.1), ylim=(0, DURATION))
    chart.axes[3].set(xlim=(-2, 2), ylim=(-2, 2))

    ax_polar=chart.figure.add_subplot(2, 2, 1, polar=True)
    dsp.Stem(df_r, target='y', feature='x', bottom=0, ax=ax_polar)()
    ax_polar.plot(df_angle['x'], df_angle['y'], 'k--', alpha=0.5)
    ax_polar.plot(df_sides['x_sin'], df_sides['y_sin'], c=dsp.COLOR.BAD)
    ax_polar.plot(df_sides['x_cos'], df_sides['y_cos'], c=dsp.COLOR.GOOD)
    ax_polar.set(yticks=[], ylim=(0, 1),)

interact(
        plot_correlation_coefficient,
        correlation_coefficient=FloatSlider(
            value=0.0, min=-1.0, max=1.0, step=0.01,
            description='R:',
            continuous_update=True,
        )
    )


interactive(children=(FloatSlider(value=0.0, description='R:', max=1.0, min=-1.0, step=0.01), Output()), _dom_…

<function __main__.plot_correlation_coefficient(correlation_coefficient: float) -> None>